In [1]:
import pandas as pd
import time
import gc

In [2]:
path = '/home/vishy/Desktop/Kaggle/Avito/Data/'
subpath = '/home/vishy/Desktop//Kaggle/Avito/Submissions/'

In [3]:
text_cols = ['param_1','param_2','param_3','title','description']
print('loading train...')
train = pd.read_csv(path+'train.csv', index_col = 'item_id', usecols = text_cols + ['item_id','image_top_1'])
train_indices = train.index
print('loading test')
test = pd.read_csv(path+'test.csv', index_col = 'item_id', usecols = text_cols + ['item_id','image_top_1'])
test_indices = test.index
print('concat dfs')
df = pd.concat([train,test])
nan_indices = df[pd.isnull(df['image_top_1'])].index
not_nan_indices = df[pd.notnull(df['image_top_1'])].index

#df = df[pd.notnull(df['image_top_1'])]

del train, test

loading train...
loading test
concat dfs


In [4]:
print(df.shape)
df.head()

(2011862, 6)


,param_1,param_2,param_3,title,description,image_top_1
item_id,,,,,,
b912c3c6a6ad,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",1008.0
2dac0150717d,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",692.0
ba83aefab5dc,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",3032.0
02996f1dd2ea,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,796.0
7c90be56d2ab,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,2264.0


In [5]:
print('cleaning text')
for col in text_cols:
    df[col] = df[col].fillna('nan').astype(str)
print('concat text')
df['text'] = df[text_cols].apply(lambda x: ' '.join(x), axis=1)
df.drop(text_cols,axis = 1, inplace = True)

cleaning text
concat text


In [6]:
df.head()

,image_top_1,text
item_id,,
b912c3c6a6ad,1008.0,Постельные принадлежности nan nan Кокоби(кокон...
2dac0150717d,692.0,Другое nan nan Стойка для Одежды Стойка для од...
ba83aefab5dc,3032.0,"Видео, DVD и Blu-ray плееры nan nan Philips bl..."
02996f1dd2ea,796.0,Автомобильные кресла nan nan Автокресло Продам...
7c90be56d2ab,2264.0,"С пробегом ВАЗ (LADA) 2110 ВАЗ 2110, 2003 Все ..."


In [7]:
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences

max_features = 100000 # max amount of words considered
max_len = 100 #maximum length of text
dim = 100 #dimension of embedding


print('tokenizing...',end='')
tic = time.time()
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['text'].values))
toc = time.time()
print('done. {}'.format(toc-tic))

col = 'text'
print("   Transforming {} to seq...".format(col))
tic = time.time()
df[col] = tokenizer.texts_to_sequences(df[col])
toc = time.time()
print('done. {}'.format(toc-tic))

print('padding X_train')
tic = time.time()
X_train = pad_sequences(df.loc[not_nan_indices,col], maxlen=max_len)
toc = time.time()
print('done. {}'.format(toc-tic))

print('padding X_nan')
tic = time.time()
X_nan = pad_sequences(df.loc[nan_indices,col], maxlen=max_len)
toc = time.time()
print('done. {}'.format(toc-tic))

df.drop(['text'], axis = 1, inplace=True)

/home/vishy/anaconda3/envs/tensorflowenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


tokenizing...done. 76.29024243354797
   Transforming text to seq...
done. 63.28343439102173
padding X_train
done. 8.635580778121948
padding X_nan
done. 0.7657039165496826


In [8]:
y = df.loc[not_nan_indices,'image_top_1'].values

In [9]:
import numpy as np
from keras.layers import Input,PReLU,BatchNormalization, GlobalMaxPooling1D, GlobalAveragePooling1D, CuDNNGRU, Bidirectional, Dense, Embedding
from keras.layers import Concatenate, Flatten, Bidirectional
from keras.optimizers import Adam
from keras.initializers import he_uniform
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping


from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy



def all_pool(tensor):
    avg_tensor = GlobalAveragePooling1D()(tensor)
    max_tensor = GlobalMaxPooling1D()(tensor)
    res_tensor = Concatenate()([avg_tensor, max_tensor])
    return res_tensor

def build_model():
    inp = Input(shape=(max_len,))

    embedding = Embedding(max_features + 1, dim)(inp)
    x = Bidirectional(CuDNNGRU(64,return_sequences=True))(embedding)
    x = all_pool(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation = 'relu')(x)
    out = Dense(3067, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)

    model.compile(optimizer=Adam(lr=0.0005), loss=sparse_categorical_crossentropy)
    return model

model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     10000100    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 128)     63744       embedding_1[0][0]                
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 128)          0           bidirectional_1[0][0]            
__________________________________________________________________________________________________
global_max

In [10]:
early_stop = EarlyStopping(patience=5)
check_point = ModelCheckpoint('model.hdf5', monitor = "val_loss", mode = "min", save_best_only = True, verbose = 1)

history = model.fit(X_train, y, batch_size = 512, epochs = 30, verbose = 1, validation_split=0.1,
                    callbacks=[early_stop,check_point])

Train on 1670998 samples, validate on 185667 samples
Epoch 1/30
1670998/1670998 [==============================] - 90s 54us/step - loss: 3.1640 - val_loss: 2.7277

Epoch 00001: val_loss improved from inf to 2.72774, saving model to model.hdf5
Epoch 2/30
1670998/1670998 [==============================] - 86s 51us/step - loss: 2.5333 - val_loss: 2.6552

Epoch 00002: val_loss improved from 2.72774 to 2.65520, saving model to model.hdf5
Epoch 3/30
1670998/1670998 [==============================] - 88s 53us/step - loss: 2.3603 - val_loss: 2.6726

Epoch 00003: val_loss did not improve from 2.65520
Epoch 4/30
1670998/1670998 [==============================] - 89s 53us/step - loss: 2.2133 - val_loss: 2.7640

Epoch 00004: val_loss did not improve from 2.65520
Epoch 5/30
1670998/1670998 [==============================] - 89s 53us/step - loss: 2.0738 - val_loss: 2.8253

Epoch 00005: val_loss did not improve from 2.65520
Epoch 6/30
1670998/1670998 [==============================] - 89s 53us/step -

In [11]:
id2word = {tokenizer.word_index[word]:word for word in tokenizer.word_index}
weights = model.layers[1].get_weights()[0]
embedding_dict = {}
for id in id2word:
    if id <= weights.shape[0]-1:
        embedding_dict[id2word[id]] = weights[id]

In [12]:
import pickle
with open('embedding_dict.p','wb') as f:
    pickle.dump(embedding_dict,f)

In [13]:
preds = model.predict(X_nan,verbose=1)

155197/155197 [==============================] - 25s 158us/step


In [15]:
k = 0
classes = np.zeros(shape=np.argmax(preds,axis = 1).shape)
for i in range(preds.shape[0]):
    if np.max(preds[i]) > 0.1:
        k+=1
        classes[i] = np.argmax(preds[i])
    else:
        classes[i] = np.nan
df.loc[nan_indices,'image_top_1'] = classes

In [17]:
df.head()

,image_top_1
item_id,
b912c3c6a6ad,1008.0
2dac0150717d,692.0
ba83aefab5dc,3032.0
02996f1dd2ea,796.0
7c90be56d2ab,2264.0


In [18]:
df.loc[train_indices].to_csv('train_image_top_1_features.csv')
df.loc[test_indices].to_csv('test_image_top_1_features.csv')